In [ ]:
import pandas as pd
import time
import matplotlib.pyplot as plt
from datetime import date, datetime
from itertools import cycle, islice
from IPython.core.interactiveshell import InteractiveShell

In [ ]:
InteractiveShell.ast_node_interactivity = "all"
timestr = time.strftime("%Y%m%d-%H%M")
pd.options.display.max_rows = 1000

In [ ]:
registration = pd.read_excel('./data/DATA_CLEANING-35990002.xlsx', skiprows=[0])
participation = pd.read_excel('./data/DATA_CLEANING-44030001.xlsx', skiprows=[0])

In [ ]:
regs = registration[[
    'Identifier',
    '41480001|Nama Lengkap (Sesuai KTP/Kartu Pelajar)',
    '44020003|Tanggal Lahir',
    '47000004|Jenis Kelamin',
    '43440001|Alamat'
]]
pcs = participation[[
    'Identifier',
    '41480002|Daftar Kode Output-Aktifitas (Pastikan ke penyelenggara)',
    '36970009|Nama Aktifitas (lihat absensi kertas/backdrop kegiatan)',
    '40580002|Organisasi penyelenggara',
    '40550002|Tanggal Aktifitas (tanggal mulai)',
]]
youth_participation = pd.merge(pcs, regs, on='Identifier')
youth_participation = youth_participation.rename(columns = {
    '41480002|Daftar Kode Output-Aktifitas (Pastikan ke penyelenggara)':'Kode Output',
    '36970009|Nama Aktifitas (lihat absensi kertas/backdrop kegiatan)':'Nama Aktivitas',
    '40580002|Organisasi penyelenggara':'Organisasi Penyelenggara',
    '40550002|Tanggal Aktifitas (tanggal mulai)':'Tanggal Aktivitas',
    '41480001|Nama Lengkap (Sesuai KTP/Kartu Pelajar)':'Nama Lengkap',
    '44020003|Tanggal Lahir':'DOB',
    '47000004|Jenis Kelamin':'Jenis Kelamin',
    '43440001|Alamat':'Alamat',
})
youth_participation['Goal Code'] = youth_participation['Kode Output'].apply(lambda x: x.split('|')[3][:7])
youth_participation['Nama Aktivitas'] = youth_participation['Nama Aktivitas'].apply(lambda x: x.title())
youth_participation['Jenis Kelamin'] = youth_participation['Jenis Kelamin'].apply(lambda x:x.upper())
youth_participation['Provinsi'] = youth_participation['Alamat'].apply(lambda x:x.split('|')[0].replace('Prov. ',''))
youth_participation['Kota / Kab'] = youth_participation['Alamat'].apply(lambda x:x.split('|')[1])
youth_participation['Nama Lengkap'] = youth_participation['Nama Lengkap'].apply(lambda x:x.title())
youth_participation['DOB'] = youth_participation['DOB'].fillna(0)

In [ ]:
def calculateAge(born):
    born = datetime.strptime(born, "%Y-%m-%d").date()
    today = date.today()
    age = today.year - born.year - ((today.month, today.day) < (born.month, born.day))
    if age < 15:
        return '< 15'
    elif 15 <= age <=29:
        return '15 - 29'
    else:
        return '29 >'

In [ ]:
youth_participation['Umur'] = youth_participation['DOB'].apply(lambda x: None if x == 0 else calculateAge(x))

In [ ]:
graph = youth_participation.groupby(['Goal Code','Organisasi Penyelenggara']).size().unstack()
colors = list(islice(cycle(['darkcyan', 'orange', 'slateblue', 'green']), None, len(graph)))
graph = graph.plot(kind='barh', stacked=True, figsize=(17, 10), zorder=2, width=.9, color=colors)
graph.grid(zorder=1, linestyle='-', linewidth=.3, alpha=.5)
graph.set_title('Partisipan disetiap kota berdasarkan Organisasi Penyelenggara dan Goal Code')
graph.set_ylabel('Goal Code')
graph.set_xlabel('Total Peserta')

In [ ]:
graph = youth_participation.groupby(['Goal Code','Jenis Kelamin']).size().unstack()
colors = list(islice(cycle(['darkcyan', 'coral']), None, len(graph)))
graph = graph.plot(kind='barh', stacked=True, figsize=(17, 10), zorder=2, width=.9, color=colors)
graph.grid(zorder=1, linestyle='-', linewidth=.3, alpha=.5)
graph.set_title('Partisipan disetiap kota berdasarkan Jenis Kelamin dan Goal Code')
graph.set_ylabel('Goal Code')
graph.set_xlabel('Total Peserta')

In [ ]:
graph = youth_participation.groupby(['Kota / Kab','Jenis Kelamin']).size().unstack()
colors = list(islice(cycle(['darkcyan', 'coral']), None, len(graph)))
graph = graph.plot(kind='barh', stacked=True, figsize=(17, 10), zorder=2, width=.9, color=colors)
graph.grid(zorder=1, linestyle='-', linewidth=.3, alpha=.5)
graph.set_title('Partisipan disetiap kota berdasarkan Jenis Kelamin')
graph.set_ylabel('Propinsi')
graph.set_ylabel('Total Peserta')

In [ ]:
group_names=['DKI Jakarta','Jawa Barat','Sulawesi Selatan','Sulawesi Tenggara']
group_size=youth_participation.groupby(['Provinsi']).size()
subgroup_names=['Laki-Laki','Perempuan']
subgroup_size=youth_participation.groupby(['Jenis Kelamin']).size()
 
fig, ax = plt.subplots()
ax.axis('equal')
mypie, _ = ax.pie(group_size, radius=1, labels=group_names, colors=['slateblue', 'tomato', 'orange', 'forestgreen'])
plt.setp( mypie, width=0.4, edgecolor='white')
 
mypie2, _ = ax.pie(subgroup_size, radius=0.6, labels=subgroup_names, labeldistance=0.7, colors=['darkcyan', 'coral'])
plt.setp( mypie2, width=0.3, edgecolor='white')
plt.title('Seluruh Aktivitas berdasarkan Jenis Kelamin')
plt.show()

In [ ]:
print('\nSeluruh Partisipan Aktivitas dengan Goal Code 1.1.1.1 berdasarkan Jenis Kelamin')
youth_participation.loc[youth_participation['Goal Code'] == '1.1.1.1'].groupby(['Jenis Kelamin']).size().to_frame('Peserta').reset_index()

print('\nSeluruh Partisipan Aktivitas dengan Goal Code 1.1.1.1 berdasarkan Usia')
youth_participation.loc[youth_participation['Goal Code'] == '1.1.1.1'].groupby(['Umur']).size().to_frame('Peserta').reset_index()

print('\nSeluruh Peserta yang Hadir di Workshop Video Berdasarkan Umur')
youth_participation.loc[youth_participation['Nama Aktivitas'].str.contains('Video')].groupby(['Umur']).size().to_frame('Peserta').reset_index()

print('\nSeluruh Peserta yang Hadir di Workshop Video')
youth_participation.loc[youth_participation['Nama Aktivitas'].str.contains('Video')].groupby(['Kota / Kab']).size().to_frame('Peserta').reset_index()

print('\nSeluruh Partisipan Aktivitas dengan Goal Code 1.1.1.1 berdasarkan Nama Aktivitas')
youth_participation.loc[youth_participation['Goal Code'] == '1.1.1.1'].groupby(['Nama Aktivitas']).size().to_frame('Peserta').reset_index()

In [ ]:
print('\nSeluruh Partisipan Aktivitas dengan Goal Code 1.1.2 berdasarkan Jenis Kelamin')
youth_participation.loc[youth_participation['Goal Code'].str.contains('1.1.2')].groupby(['Jenis Kelamin']).size().to_frame('Peserta').reset_index()

print('\nSeluruh Partisipan Aktivitas dengan Goal Code 1.1.2 berdasarkan Usia')
youth_participation.loc[youth_participation['Goal Code'].str.contains('1.1.2')].groupby(['Umur']).size().to_frame('Peserta').reset_index()

print('\nSeluruh Peserta yang Hadir di Workshop Video Berdasarkan Umur')
youth_participation.loc[youth_participation['Nama Aktivitas'].str.contains('Video')].groupby(['Umur']).size().to_frame('Peserta').reset_index()

print('\nSeluruh Peserta yang Hadir di Workshop Video')
youth_participation.loc[youth_participation['Nama Aktivitas'].str.contains('Video')].groupby(['Kota / Kab']).size().to_frame('Peserta').reset_index()

print('\nSeluruh Partisipan Aktivitas dengan Goal Code 1.1.2 berdasarkan Nama Aktivitas')
youth_participation.loc[youth_participation['Goal Code'].str.contains('1.1.2')].groupby(['Nama Aktivitas']).size().to_frame('Peserta').reset_index()